In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(style="ticks")
import matplotlib.pyplot as plt
import multiprocessing
from joblib import Parallel, delayed
%matplotlib inline
from scipy.optimize import curve_fit 
from scipy import stats
import datetime as dt
import statsmodels.api as sm
from pandas.plotting import scatter_matrix
from sklearn.linear_model import LinearRegression
from scipy.stats import skew
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
import pickle

### Loading raw data into data frame

In [2]:
##Path to the raw data
#path="/home/d.dasarathan/project_sml/df_raw_data.pkl"
#path ='/home/d.dasarathan/project_sml/100Kdataset/df_raw_100k.pkl'
path ='/home/d.dasarathan/project_sml/100Kdataset/df_raw_600k.pkl'

In [3]:
path ='/home/d.dasarathan/project_sml/100Kdataset/df_raw_600k.pkl'
df_raw_data=pd.read_pickle(path)
df_raw_data.head()

,Account,AdminComment,AllocCPUS,AllocGRES,AllocNodes,AllocTRES,AssocID,AveCPU,AveCPUFreq,AveDiskRead,AveDiskWrite,AvePages,AveRSS,AveVMSize,BlockID,Cluster,Comment,ConsumedEnergy,ConsumedEnergyRaw,CPUTime,CPUTimeRAW,DerivedExitCode,Elapsed,ElapsedRaw,Eligible,End,ExitCode,GID,Group,JobID,JobIDRaw,JobName,Layout,MaxDiskRead,MaxDiskReadNode,MaxDiskReadTask,MaxDiskWrite,MaxDiskWriteNode,MaxDiskWriteTask,MaxPages,MaxPagesNode,MaxPagesTask,MaxRSS,MaxRSSNode,MaxRSSTask,MaxVMSize,MaxVMSizeNode,MaxVMSizeTask,McsLabel,MinCPU,MinCPUNode,MinCPUTask,NCPUS,NNodes,NodeList,NTasks,Priority,Partition,QOS,QOSRAW,ReqCPUFreq,ReqCPUFreqMin,ReqCPUFreqMax,ReqCPUFreqGov,ReqCPUS,ReqGRES,ReqMem,ReqNodes,ReqTRES,Reservation,ReservationId,Reserved,ResvCPU,ResvCPURAW,Start,State,Submit,Suspended,SystemCPU,Timelimit,TotalCPU,UID,User,UserCPU,WCKey,WCKeyID,WorkDir,Unnamed: 87
0,f.hung,NaN,28.0,NaN,1.0,"cpu=28,mem=56000M,node=1,billing=28",2814,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discovery,NaN,0.0,0.0,3429-08:10:20,296295020.0,0:0,122-11:26:05,10581965,2019-10-25T16:32:05,2020-02-25T02:59:10,0:0,100.0,users,7467545,7467545,mmtcncwi_yzD_md,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28,1.0,c4035,NaN,4245.0,hung,normal,1.0,Unknown,Unknown,Unknown,Unknown,28.0,NaN,2000Mc,1,"cpu=28,mem=56000M,node=1",NaN,NaN,00:01:00,00:28:00,1680.0,2019-10-25T16:33:05,COMPLETED,2019-10-25T16:32:05,00:00:00,1-05:14:21,UNLIMITED,122-08:19:38,5.069700e+04,vodopivec.a,121-03:05:17,NaN,0.0,/scratch/vodopivec.a/MMT_CNC_YZ_scat_MD,NaN
3,f.hung,NaN,28.0,NaN,1.0,"cpu=28,mem=56000M,node=1,billing=28",2814,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discovery,NaN,0.0,0.0,3535-12:08:00,305467680.0,0:0,126-06:26:00,10909560,2019-10-25T16:34:09,2020-02-28T22:01:07,0:0,100.0,users,7467547,7467547,YZmmtcncwi_yz_md,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28,1.0,c3161,NaN,4245.0,hung,normal,1.0,Unknown,Unknown,Unknown,Unknown,28.0,NaN,2000Mc,1,"cpu=28,mem=56000M,node=1",NaN,NaN,00:00:58,00:27:04,1624.0,2019-10-25T16:35:07,COMPLETED,2019-10-25T16:34:09,00:00:00,1-08:48:14,UNLIMITED,126-03:57:06,5.069700e+04,vodopivec.a,124-19:08:52,NaN,0.0,/scratch/vodopivec.a/MMT2_CNC_YZ_MD,NaN
6,allshouse,NaN,8.0,NaN,0.0,NaN,2755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discovery,***TEST_COMMENT***,NaN,NaN,00:00:00,0.0,0:0,00:00:00,0,2019-11-19T11:38:41,2020-02-03T11:01:22,0:0,100.0,users,7780464,7780464,wind_interp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,1.0,None assigned,NaN,6298.0,interactive,normal,1.0,Unknown,Unknown,Unknown,Unknown,8.0,NaN,100Gn,1,"cpu=8,mem=100G,node=1",NaN,NaN,75-23:22:41,607-19:01:28,52513288.0,2020-02-03T11:01:22,NODE_FAIL,2019-11-19T11:38:41,00:00:00,NaN,08:00:00,00:00:00,1.825241e+09,fennessy.fr,NaN,NaN,0.0,/scratch/fennessy.fr/wind,NaN
7,sioannidis,NaN,80.0,NaN,5.0,"cpu=80,mem=160000M,node=5,billing=80",3177,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discovery,NaN,0.0,0.0,00:24:00,1440.0,1:0,00:00:18,18,2019-11-26T14:23:28,2020-02-03T15:34:27,1:0,100.0,users,7848811,7848811,workers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80,5.0,c[2137-2141],NaN,4355.0,multigpu,normal,1.0,Unknown,Unknown,Unknown,Unknown,5.0,NaN,2000Mc,5,"cpu=5,mem=10000M,node=5",NaN,NaN,69-01:10:41,345-05:53:25,29829205.0,2020-02-03T15:34:09,FAILED,2019-11-26T14:23:28,00:00:00,00:00.721,04:00:00,00:05.179,1.825455e+09,jia.wa,00:04.458,NaN,0.0,/scratch/jia.wa/spark,NaN
11,j.dy,NaN,1.0,NaN,1.0,"cpu=1,mem=16G,node=1,billing=1",3065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discovery,***TEST_COMMENT***,0.0,0.0,1-00:00:11,86411.0,0:0,1-00:00:11,86411,2020-01-02T12:30:27,2020-02-04T15:49:52,0:0,100.0,users,8421446_1,8421446,Traj,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,c0296,NaN,23866.0,general,normal,1.0,Unknown,Unknown,Unknown,Unknown,1.0,NaN,16Gn,1,"cpu=1,mem=16G,node=1",NaN,NaN,32-03:19:14,32-03:19:14,2776754.0,2020-02-03T15:49:41,TIMEOUT,2020-01-02T12:30:27,00:0

In [6]:
df_raw_data.columns

Index(['Account', 'AdminComment', 'AllocCPUS', 'AllocGRES', 'AllocNodes',
       'AllocTRES', 'AssocID', 'AveCPU', 'AveCPUFreq', 'AveDiskRead',
       'AveDiskWrite', 'AvePages', 'AveRSS', 'AveVMSize', 'BlockID', 'Cluster',
       'Comment', 'ConsumedEnergy', 'ConsumedEnergyRaw', 'CPUTime',
       'CPUTimeRAW', 'DerivedExitCode', 'Elapsed', 'ElapsedRaw', 'Eligible',
       'End', 'ExitCode', 'GID', 'Group', 'JobID', 'JobIDRaw', 'JobName',
       'Layout', 'MaxDiskRead', 'MaxDiskReadNode', 'MaxDiskReadTask',
       'MaxDiskWrite', 'MaxDiskWriteNode', 'MaxDiskWriteTask', 'MaxPages',
       'MaxPagesNode', 'MaxPagesTask', 'MaxRSS', 'MaxRSSNode', 'MaxRSSTask',
       'MaxVMSize', 'MaxVMSizeNode', 'MaxVMSizeTask', 'McsLabel', 'MinCPU',
       'MinCPUNode', 'MinCPUTask', 'NCPUS', 'NNodes', 'NodeList', 'NTasks',
       'Priority', 'Partition', 'QOS', 'QOSRAW', 'ReqCPUFreq', 'ReqCPUFreqMin',
       'ReqCPUFreqMax', 'ReqCPUFreqGov', 'ReqCPUS', 'ReqGRES', 'ReqMem',
       'ReqNodes', 'ReqTRES', 

In [4]:
df_raw_data.shape

(600000, 88)

In [4]:
df_raw_data.head(-1)

,Account,AdminComment,AllocCPUS,AllocGRES,AllocNodes,AllocTRES,AssocID,AveCPU,AveCPUFreq,AveDiskRead,AveDiskWrite,AvePages,AveRSS,AveVMSize,BlockID,Cluster,Comment,ConsumedEnergy,ConsumedEnergyRaw,CPUTime,CPUTimeRAW,DerivedExitCode,Elapsed,ElapsedRaw,Eligible,End,ExitCode,GID,Group,JobID,JobIDRaw,JobName,Layout,MaxDiskRead,MaxDiskReadNode,MaxDiskReadTask,MaxDiskWrite,MaxDiskWriteNode,MaxDiskWriteTask,MaxPages,MaxPagesNode,MaxPagesTask,MaxRSS,MaxRSSNode,MaxRSSTask,MaxVMSize,MaxVMSizeNode,MaxVMSizeTask,McsLabel,MinCPU,MinCPUNode,MinCPUTask,NCPUS,NNodes,NodeList,NTasks,Priority,Partition,QOS,QOSRAW,ReqCPUFreq,ReqCPUFreqMin,ReqCPUFreqMax,ReqCPUFreqGov,ReqCPUS,ReqGRES,ReqMem,ReqNodes,ReqTRES,Reservation,ReservationId,Reserved,ResvCPU,ResvCPURAW,Start,State,Submit,Suspended,SystemCPU,Timelimit,TotalCPU,UID,User,UserCPU,WCKey,WCKeyID,WorkDir,Unnamed: 87
0,steven.lopez,NaN,7.0,NaN,1.0,"cpu=7,mem=55G,node=1,billing=7",5187,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discovery,NaN,0.0,0.0,00:40:36,2436.0,0:0,00:05:48,348,2020-05-17T15:35:54,2020-05-17T16:01:36,0:0,100.0,users,11602102_128,11602922,cp-4e4r-ORCA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,1.0,d3004,NaN,16007.0,lopez,normal,1.0,Unknown,Unknown,Unknown,Unknown,7.0,NaN,55Gn,1,"cpu=7,mem=55G,node=1",NaN,NaN,00:19:54,02:19:18,8358.0,2020-05-17T15:55:48,COMPLETED,2020-05-17T15:35:52,00:00:00,00:09.466,1-00:00:00,29:29.533,1.825619e+09,steinr,29:20.066,NaN,0.0,/scratch/steinr/ORCA/tod/opt/cp-4e4r/CREST,NaN
1,lotterhos,NaN,1.0,NaN,1.0,"cpu=1,mem=50G,node=1,billing=1",4488.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discovery,NaN,0.0,0.0,00:01:35,95.0,0:0,00:01:35,95,2020-08-13T17:14:24,2020-08-13T17:21:38,0:0,100.0,users,13253497,13253497,3383.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,c0198,NaN,17612.0,express,normal,1.0,Unknown,Unknown,Unknown,Unknown,1.0,NaN,50Gn,1.0,"cpu=1,mem=50G,node=1",NaN,NaN,00:05:39,00:05:39,339.0,2020-08-13T17:20:03,COMPLETED,2020-08-13T17:14:24,00:00:00,00:00.540,01:00:00,01:34.502,1.825568e+09,albecker,01:33.961,NaN,0.0,/scratch/albecker/Power_analysis,NaN
2,dapeng,NaN,28.0,NaN,1.0,"cpu=28,mem=200G,node=1,billing=28",5831,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discovery,NaN,0.0,0.0,00:22:52,1372.0,0:0,00:00:49,49,2020-09-08T00:40:55,2020-09-08T00:41:58,0:0,100.0,users,13946485,13946485,short_22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28,1.0,c0476,NaN,15077.0,short,normal,1.0,Unknown,Unknown,Unknown,Unknown,28.0,NaN,200Gn,1,"cpu=28,mem=200G,node=1",NaN,NaN,00:00:14,00:06:32,392.0,2020-09-08T00:41:09,COMPLETED,2020-09-08T00:40:55,00:00:00,00:02.827,1-00:00:00,00:18.500,1.825455e+09,huang.junx,00:15.673,NaN,0.0,/scratch/jxh,NaN
3,steven.lopez,NaN,1.0,NaN,1.0,"cpu=1,mem=3G,node=1,billing=1",4525.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discovery,NaN,0.0,0.0,00:07:29,449.0,0:0,00:07:29,449,2020-08-06T17:10:51,2020-08-06T17:31:15,0:0,100.0,users,13030602,13030602,2bt-83-81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,c0308,NaN,15738.0,short,normal,1.0,Unknown,Unknown,Unknown,Unknown,1.0,NaN,3Gn,1.0,"cpu=1,mem=3G,node=1",NaN,NaN,00:12:55,00:12:55,775.0,2020-08-06T17:23:46,COMPLETED,2020-08-06T17:10:51,00:00:00,00:59.926,10:00:00,07:19.510,1.825607e+09,lijingbai2009,06:19.584,NaN,0.0,/scratch/lijingbai2009/NN-PES-QD/training-data...,NaN
4,netsi,NaN,1.0,NaN,1.0,"cpu=1,mem=200G,node=1,billing=1",5217,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discovery,NaN,0.0,0.0,00:00:03,3.0,0:0,00:00:03,3,2020-09-11T19:11:53,2020-09-13T02:21:24,1:0,100.0,users,14255256,14255256,un_eval,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,c3201,NaN,16757.0,netsi_standard,normal,1.0,Unknown,Unknown,Unknown,Unknown,1.0,NaN,200Gn,1,"cpu=1,mem=200G,node=1",NaN,NaN,1-07:09:28,1-07:09:28,112168.0,2020-09-13T02:21:21,FAILED,2020-09-11T19:11:53,00:00:00,00:00.523,5-00:00:00,00:01.382,5.118800e+04,gallagher.r,00:00.858,NaN,0.0,/sc

In [5]:
print('Submit')
print('Min: ',df_raw_data['Submit'].min(axis=0))
print('Max: ',df_raw_data['Submit'].max(axis=0))

Submit
Min:  2019-11-05T15:12:16
Max:  2020-11-23T23:55:50


In [6]:
print('End')
print('Min: ',df_raw_data['End'].min(axis=0))
print('Max: ',df_raw_data['End'].max(axis=0))

End
Min:  2020-02-01T00:00:13
Max:  Unknown


In [7]:
print (f"There are total {df_raw_data.shape[0]} jobs")
print ("Total unique users ",len(df_raw_data['User'].unique()))

There are total 500000 jobs
Total unique users  764


In [8]:
df_raw_data.describe()

,AllocCPUS,AllocNodes,ConsumedEnergy,CPUTimeRAW,MaxDiskWriteTask,NNodes,Priority,QOSRAW,ReqCPUS,ReservationId,ResvCPURAW,UID,WCKeyID,Unnamed: 87
count,500000.000000,500000.000000,478312.0,5.000000e+05,0.0,500000.000000,500000.000000,500000.000000,500000.000000,4596.000000,5.000000e+05,4.973010e+05,500000.0,0.0
mean,17.792502,1.160694,0.0,7.113622e+04,NaN,1.207486,13526.352586,1.000038,16.408178,190.710183,2.006599e+05,8.325755e+08,0.0,NaN
std,34.864461,1.242722,0.0,8.759035e+05,NaN,1.249302,5822.048627,0.026870,32.585924,29.886160,2.107648e+07,9.092096e+08,0.0,NaN
min,1.000000,0.000000,0.0,0.000000e+00,NaN,1.000000,1.000000,1.000000,1.000000,127.000000,0.000000e+00,0.000000e+00,0.0,NaN
25%,1.000000,1.000000,0.0,8.500000e+01,NaN,1.000000,13871.000000,1.000000,1.000000,183.000000,5.300000e+01,5.092800e+04,0.0,NaN
50%,7.000000,1.000000,0.0,5.720000e+02,NaN,1.000000,15724.000000,1.000000,4.000000,183.000000,5.880000e+02,5.128500e+04,0.0,NaN
75%,28.000000,1.000000,0.0,6.197000e+03,NaN,1.000000,16498.000000,1.000000,28.000000,216.000000,5.214000e+03,1.825548e+09,0.0,NaN
max,3136.000000,256.000000,0.0,2.832782e+08,NaN,256.000000,23732.000000,20.000000,3136.000000,252.000000,4.294967e+09,1.825729e+09,0.0,NaN


In [9]:
df_raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 88 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Account            500000 non-null  object 
 1   AdminComment       0 non-null       object 
 2   AllocCPUS          500000 non-null  float64
 3   AllocGRES          16962 non-null   object 
 4   AllocNodes         500000 non-null  float64
 5   AllocTRES          479025 non-null  object 
 6   AssocID            500000 non-null  object 
 7   AveCPU             0 non-null       object 
 8   AveCPUFreq         0 non-null       object 
 9   AveDiskRead        0 non-null       object 
 10  AveDiskWrite       0 non-null       object 
 11  AvePages           0 non-null       object 
 12  AveRSS             0 non-null       object 
 13  AveVMSize          0 non-null       object 
 14  BlockID            0 non-null       object 
 15  Cluster            500000 non-null  object 
 16  Co

### Data Cleaning

#### Remove the letter 'T' from the timestamp columns

In [10]:
df_raw_data['Submit']=df_raw_data['Submit'].str.replace('T', ' ')
df_raw_data['End']=df_raw_data['End'].str.replace('T', ' ')
df_raw_data['Start']=df_raw_data['Start'].str.replace('T', ' ')
df_raw_data['Eligible']=df_raw_data['Eligible'].str.replace('T', ' ')

#### Replace unknown with NaN

In [11]:
print("Submit column has {} unknown values".format((df_raw_data.Submit == 'Unknown').sum()))
print("Eligible column has {} unknown values".format((df_raw_data.Eligible == 'Unknown').sum()))
print("Start column has {} unknown values".format((df_raw_data.Start == 'Unknown').sum()))
print("End column has {} unknown values".format((df_raw_data.End == 'Unknown').sum()))

print("******Replacing unknown values with NaN******")
df_raw_data['Submit']=df_raw_data["Submit"].replace({"Unknown": np.nan})
df_raw_data['Eligible']=df_raw_data["Eligible"].replace({"Unknown": np.nan})
df_raw_data['Start']=df_raw_data["Start"].replace({"Unknown": np.nan})
df_raw_data["End"]=df_raw_data["End"].replace({"Unknown": np.nan})

print("Submit column has {} unknown values".format((df_raw_data.Submit == 'Unknown').sum()))
print("Eligible column has {} unknown values".format((df_raw_data.Eligible == 'Unknown').sum()))
print("Start column has {} unknown values".format((df_raw_data.Start == 'Unknown').sum()))
print("End column has {} unknown values".format((df_raw_data.End == 'Unknown').sum()))

Submit column has 0 unknown values
Eligible column has 0 unknown values
Start column has 1 unknown values
End column has 98 unknown values
******Replacing unknown values with NaN******
Submit column has 0 unknown values
Eligible column has 0 unknown values
Start column has 0 unknown values
End column has 0 unknown values


#### Drop samples that has invalid or NaN in End Column i.e the job was still runnning at the time of pulling data from SLURM
#### Also drop samples that has end time later 2021

In [12]:
df_raw_data.drop(df_raw_data[(df_raw_data['End'].str.contains("NaN")==False) & (df_raw_data['End'].str.contains("20")==False)].index, inplace=True)
df_raw_data.drop(df_raw_data[(df_raw_data['Eligible'].str.contains("NaN")==False) & (df_raw_data['Eligible'].str.contains("20")==False)].index, inplace=True)

#### Change datatype for timestamp

In [13]:
df_raw_data['Submit']=pd.to_datetime(df_raw_data['Submit'])
df_raw_data['Start']=pd.to_datetime(df_raw_data['Start'])
df_raw_data['End']=pd.to_datetime(df_raw_data['End'])
df_raw_data['Eligible']=pd.to_datetime(df_raw_data['Eligible'])
df_raw_data.head()

,Account,AdminComment,AllocCPUS,AllocGRES,AllocNodes,AllocTRES,AssocID,AveCPU,AveCPUFreq,AveDiskRead,AveDiskWrite,AvePages,AveRSS,AveVMSize,BlockID,Cluster,Comment,ConsumedEnergy,ConsumedEnergyRaw,CPUTime,CPUTimeRAW,DerivedExitCode,Elapsed,ElapsedRaw,Eligible,End,ExitCode,GID,Group,JobID,JobIDRaw,JobName,Layout,MaxDiskRead,MaxDiskReadNode,MaxDiskReadTask,MaxDiskWrite,MaxDiskWriteNode,MaxDiskWriteTask,MaxPages,MaxPagesNode,MaxPagesTask,MaxRSS,MaxRSSNode,MaxRSSTask,MaxVMSize,MaxVMSizeNode,MaxVMSizeTask,McsLabel,MinCPU,MinCPUNode,MinCPUTask,NCPUS,NNodes,NodeList,NTasks,Priority,Partition,QOS,QOSRAW,ReqCPUFreq,ReqCPUFreqMin,ReqCPUFreqMax,ReqCPUFreqGov,ReqCPUS,ReqGRES,ReqMem,ReqNodes,ReqTRES,Reservation,ReservationId,Reserved,ResvCPU,ResvCPURAW,Start,State,Submit,Suspended,SystemCPU,Timelimit,TotalCPU,UID,User,UserCPU,WCKey,WCKeyID,WorkDir,Unnamed: 87
0,steven.lopez,NaN,7.0,NaN,1.0,"cpu=7,mem=55G,node=1,billing=7",5187,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discovery,NaN,0.0,0.0,00:40:36,2436.0,0:0,00:05:48,348,2020-05-17 15:35:54,2020-05-17 16:01:36,0:0,100.0,users,11602102_128,11602922,cp-4e4r-ORCA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,1.0,d3004,NaN,16007.0,lopez,normal,1.0,Unknown,Unknown,Unknown,Unknown,7.0,NaN,55Gn,1,"cpu=7,mem=55G,node=1",NaN,NaN,00:19:54,02:19:18,8358.0,2020-05-17 15:55:48,COMPLETED,2020-05-17 15:35:52,00:00:00,00:09.466,1-00:00:00,29:29.533,1.825619e+09,steinr,29:20.066,NaN,0.0,/scratch/steinr/ORCA/tod/opt/cp-4e4r/CREST,NaN
1,lotterhos,NaN,1.0,NaN,1.0,"cpu=1,mem=50G,node=1,billing=1",4488.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discovery,NaN,0.0,0.0,00:01:35,95.0,0:0,00:01:35,95,2020-08-13 17:14:24,2020-08-13 17:21:38,0:0,100.0,users,13253497,13253497,3383.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,c0198,NaN,17612.0,express,normal,1.0,Unknown,Unknown,Unknown,Unknown,1.0,NaN,50Gn,1.0,"cpu=1,mem=50G,node=1",NaN,NaN,00:05:39,00:05:39,339.0,2020-08-13 17:20:03,COMPLETED,2020-08-13 17:14:24,00:00:00,00:00.540,01:00:00,01:34.502,1.825568e+09,albecker,01:33.961,NaN,0.0,/scratch/albecker/Power_analysis,NaN
2,dapeng,NaN,28.0,NaN,1.0,"cpu=28,mem=200G,node=1,billing=28",5831,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discovery,NaN,0.0,0.0,00:22:52,1372.0,0:0,00:00:49,49,2020-09-08 00:40:55,2020-09-08 00:41:58,0:0,100.0,users,13946485,13946485,short_22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28,1.0,c0476,NaN,15077.0,short,normal,1.0,Unknown,Unknown,Unknown,Unknown,28.0,NaN,200Gn,1,"cpu=28,mem=200G,node=1",NaN,NaN,00:00:14,00:06:32,392.0,2020-09-08 00:41:09,COMPLETED,2020-09-08 00:40:55,00:00:00,00:02.827,1-00:00:00,00:18.500,1.825455e+09,huang.junx,00:15.673,NaN,0.0,/scratch/jxh,NaN
3,steven.lopez,NaN,1.0,NaN,1.0,"cpu=1,mem=3G,node=1,billing=1",4525.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discovery,NaN,0.0,0.0,00:07:29,449.0,0:0,00:07:29,449,2020-08-06 17:10:51,2020-08-06 17:31:15,0:0,100.0,users,13030602,13030602,2bt-83-81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,c0308,NaN,15738.0,short,normal,1.0,Unknown,Unknown,Unknown,Unknown,1.0,NaN,3Gn,1.0,"cpu=1,mem=3G,node=1",NaN,NaN,00:12:55,00:12:55,775.0,2020-08-06 17:23:46,COMPLETED,2020-08-06 17:10:51,00:00:00,00:59.926,10:00:00,07:19.510,1.825607e+09,lijingbai2009,06:19.584,NaN,0.0,/scratch/lijingbai2009/NN-PES-QD/training-data...,NaN
4,netsi,NaN,1.0,NaN,1.0,"cpu=1,mem=200G,node=1,billing=1",5217,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discovery,NaN,0.0,0.0,00:00:03,3.0,0:0,00:00:03,3,2020-09-11 19:11:53,2020-09-13 02:21:24,1:0,100.0,users,14255256,14255256,un_eval,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,c3201,NaN,16757.0,netsi_standard,normal,1.0,Unknown,Unknown,Unknown,Unknown,1.0,NaN,200Gn,1,"cpu=1,mem=200G,node=1",NaN,NaN,1-07:09:28,1-07:09:28,112168.0,2020-09-13 02:21:21,FAILED,2020-09-11 19:11:53,00:00:00,00:00.523,5-00:00:00,00:01.382,5.118800e+04,gallagher.r,00:00.858,NaN,0.0,/sc

#### Drop rows where the job started before Submit time (cannot happen in a valid scenario)

In [14]:
df_raw_data.drop(df_raw_data[df_raw_data.Start < df_raw_data.Submit].index, inplace = True) 

#### Drop all the records where the job never started or never ended

In [15]:
df_raw_data.drop(df_raw_data.loc[df_raw_data.Start.isnull()].index, inplace=True)

In [16]:
# Replacing infinite with nan
df_raw_data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [17]:
# Reset index
df_cleaned_data=df_raw_data.reset_index(drop=True)

In [18]:
# Write the cleaned dataset to a pickle file
#df_cleaned_data.to_pickle('/home/d.dasarathan/project_sml/df_cleaned_data_nonulls.pkl')
#df_cleaned_data.to_pickle('/home/d.dasarathan/project_sml/100Kdataset/df_cleaned_data_nonulls.pkl')
#df_cleaned_data.to_pickle('/home/d.dasarathan/project_sml/100Kdataset/df_cleaned_data_nonulls_500k.pkl')
df_cleaned_data.to_pickle('/home/d.dasarathan/project_sml/100Kdataset/df_cleaned_data_nonulls_600k.pkl')